## **Scraping Google Jime**

URL="https://www.google.com/about/careers/applications/jobs/results/?utm_campaign=profilepage&utm_medium=profilepage&utm_source=linkedin&src=Online%2FLinkedIn%2Flinkedin_page&location=Mexico"

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np
import re
from deep_translator import GoogleTranslator
from nanoid import generate
from slugify import slugify

In [4]:
url="https://www.google.com/about/careers/applications/jobs/results/"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Encuentra todos los enlaces que dirigen a las vacantes
    vacancy_links = []

    for link in soup.find_all("a", class_="WpHeLc VfPpkd-mRLv6 VfPpkd-RLmnJb", href=True):
        vacancy_links.append(link["href"])

    # Busco la pagina sigruiente e itero sobre los links para guardarlos en las vacantes
    page=soup.find('a', class_="WpHeLc VfPpkd-mRLv6", href=True)
    if page is not None:
        page_response=requests.get(page['href'])
        
        if page_response.status_code == 200:
            pagesoup = BeautifulSoup(page_response.text, 'html.parser')
            
            for link in pagesoup.find_all("a", class_="WpHeLc VfPpkd-mRLv6 VfPpkd-RLmnJb", href=True):
                vacancy_links.append(link["href"])
        else:
            print("No existen más páginas:", page_response.status_code)
    else:
        print("No se encontró el enlace a la página siguiente")
            
    # Los enlaces pueden ser relativos, así que construye las URL completas
    base_url = "https://www.google.com/about/careers/applications/"
    full_vacancy_links = [base_url + link for link in vacancy_links]
        
    # Crear un DataFrame a partir de la lista de enlaces
    df_url = pd.DataFrame({"url": full_vacancy_links})

else:
    print("No se pudo acceder a la página")

In [5]:
df_url.head(20)

,url
0,https://www.google.com/about/careers/applicati...
1,https://www.google.com/about/careers/applicati...
2,https://www.google.com/about/careers/applicati...
3,https://www.google.com/about/careers/applicati...
4,https://www.google.com/about/careers/applicati...
5,https://www.google.com/about/careers/applicati...
6,https://www.google.com/about/careers/applicati...
7,https://www.google.com/about/careers/applicati...
8,https://www.google.com/about/careers/applicati...
9,https://www.google.com/about/careers/applicati...


In [6]:
def ofertas_trabajo(url):
    job_offers=[]
    try:
        # realizo el request a la pagina web de trabajo y obtengo los links que contienen las categorias
        response = requests.get(url)
        
        # verifico si  la peticion fue exitosa
        if response.status_code  == 200:
            # parseo el html de la pagina con BeautifulSoup y pido que me guarde el texto
            soup = BeautifulSoup(response.content,"html.parser")
                
            title = soup.find("div", class_="sPeqm").text.strip()
            location=soup.find("span", class_= "r0wTof").text.strip()
            requirements= soup.find("div",class_="KwJkGe").text.strip()
            responsabilities= soup.find("div", class_="BDNOWe").text.strip()
            about=soup.find("div", class_="aG5W3").text.strip()
                
            jobs = {
                "title": title, 
                "location": location, 
                "requirements": requirements, 
                "responsanilities": responsabilities, 
                "about": about,
                "URL":url
            }

            # agrego cada oferta de trabajo a mi lista de ofertas
            job_offers.append(jobs)

        df = pd.DataFrame(job_offers)
        return df
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Aplicar la función a cada URL y guardar los resultados en una lista de DataFrames
resultados = []

for url in df_url['url']:
    detalles_vacante = ofertas_trabajo(url)
    if detalles_vacante is not None:
        resultados.append(detalles_vacante)

# Concatenar los DataFrames en uno solo
resultados_df = pd.concat(resultados, ignore_index=True)


In [7]:
resultados_df.head(20)

,title,location,requirements,responsanilities,about,URL
0,"Account Development Manager, Google Cloud (Eng...","Tokyo, Japan",info_outlineXGoogle welcomes people with disab...,Responsibilities\nMaximize pipeline generation...,About the jobThe Google Cloud Platform team he...,https://www.google.com/about/careers/applicati...
1,Senior System Software and Performance Archite...,"Tokyo, Japan",info_outlineXGoogle welcomes people with disab...,Responsibilities\nLead the development of meth...,About the jobOur computational challenges are ...,https://www.google.com/about/careers/applicati...
2,"Retail Partner Manager, Consumer Go-To-Market,...","Tokyo, Japan",info_outlineXA person with disabilities will b...,Responsibilities\nLead all aspects of relation...,About the jobGoogle's line of products and ser...,https://www.google.com/about/careers/applicati...
3,"Data Center Operations Facility Technician, Ge...","Tokyo, Japan",Minimum qualifications:\nTrade school qualific...,"Responsibilities\nTest, validate, improve, and...",About the jobThe Data Center team designs and ...,https://www.google.com/about/careers/applicati...
4,"Technical Solutions Consultant, Android Auto P...","Tokyo, Japan",Minimum qualifications:\nBachelor’s degree in ...,Responsibilities\nManage technical projects wi...,About the jobThe Android Partner Engineering t...,https://www.google.com/about/careers/applicati...
5,"Customer Engineering, Partner Engineershare li...","Tokyo, Japan",info_outlineXNote: By applying to this positio...,Responsibilities\nEnable system integrator (SI...,About the jobWhen leading companies choose Goo...,https://www.google.com/about/careers/applicati...
6,Lead CPU and Performance Architectshare linkCo...,"Tokyo, Japan",info_outlineXNote: By applying to this positio...,Responsibilities\nPlan and evaluate ARM’s arch...,About the jobAs a Lead CPU and Performance Arc...,https://www.google.com/about/careers/applicati...
7,"Solution Design Manager, Workforce Solutionssh...","Tokyo, Japan",Minimum qualifications:\nBachelor's degree or ...,Responsibilities\nServe as a thought partner a...,About the jobA problem isn’t truly solved unti...,https://www.google.com/about/careers/applicati...
8,"App Review Analyst, Trust and Safetyshare link...","Tokyo, Japan",Minimum qualifications:\nBachelor's degree or ...,Responsibilities\nDemonstrate excellence in de...,About the jobTrust & Safety team members are t...,https://www.google.com/about/careers/applicati...
9,"Software Engineering Manager, System Architect...","Tokyo, Japan",info_outlineXGoogle welcomes people with disab...,Responsibilities\nDeploy and optimize the next...,"About the jobLike Google's own ambitions, the ...",https://www.google.com/about/careers/applicati...
